### 네이버 부동산에서 시도, 군구, 동, 아파트명, 거래유형을 정의하면 해당 물건을 찾아주는 프로그램
### 두개의 부분으로 나누어져 있며, main 부분에 가격과 평형을 추가로 정의할 수 있음

In [45]:
import requests
import time
import pandas as pd

def get_region_list(cortarNo):
    url = f"https://m.land.naver.com/map/getRegionList?cortarNo={cortarNo}&mycortarNo="
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Referer": "https://m.land.naver.com/map/"
    }

    for attempt in range(5):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            return data['result']['list']
        except requests.exceptions.RequestException as e:
            print(f"HTTP 요청 오류 발생: {str(e)} (시도 {attempt + 1}/5)")
            time.sleep(2 ** attempt)
        except ValueError:
            print("JSON 디코딩 오류 발생")
            return []
    return []

def get_apartment_complexes_in_dong(dong_code):
    url = f"https://m.land.naver.com/complex/ajax/complexListByCortarNo?cortarNo={dong_code}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Referer": "https://m.land.naver.com/map/"
    }

    for attempt in range(5):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            return data['result']
        except requests.exceptions.RequestException as e:
            print(f"HTTP 요청 오류 발생: {str(e)} (시도 {attempt + 1}/5)")
            time.sleep(2 ** attempt)
        except ValueError:
            print("JSON 디코딩 오류 발생")
            return []
    return []

def find_hscpNo(sido, gungu, dong, apartment_name):
    regions = get_region_list("0000000000")
    
    sido_cortarNo = next((region['CortarNo'] for region in regions if region['CortarNm'] == sido), None)
    if not sido_cortarNo:
        print(f"{sido} 정보를 찾을 수 없습니다.")
        return None, None
    
    districts = get_region_list(sido_cortarNo)
    gungu_cortarNo = next((district['CortarNo'] for district in districts if district['CortarNm'] == gungu), None)
    if not gungu_cortarNo:
        print(f"{gungu} 정보를 찾을 수 없습니다.")
        return None, None
    
    dongs = get_region_list(gungu_cortarNo)
    dong_cortarNo = next((d['CortarNo'] for d in dongs if d['CortarNm'] == dong), None)
    if not dong_cortarNo:
        print(f"{dong} 정보를 찾을 수 없습니다.")
        return None, None
    
    complexes = get_apartment_complexes_in_dong(dong_cortarNo)
    complex = next((comp for comp in complexes if comp['hscpNm'] == apartment_name), None)
    if complex:
        return complex['hscpNo'], dong_cortarNo
    
    print(f"{apartment_name} 정보를 찾을 수 없습니다.")
    return None, None

def get_trade_list(hscpNo, cortarNo, tradTpCd):
    url = f"https://m.land.naver.com/complex/getComplexArticleList?hscpNo={hscpNo}&cortarNo={cortarNo}&tradTpCd={tradTpCd}&order=point_&showR0=N&page=1"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Referer": "https://m.land.naver.com/complex/"
    }
    all_trade_list = []

    page = 1
    while True:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            trade_list = data['result']['list']
            all_trade_list.extend(trade_list)
            if len(trade_list) < 20:  # If less than 20 items are returned, it's the last page
                break
            else:
                page += 1
                url = f"https://m.land.naver.com/complex/getComplexArticleList?hscpNo={hscpNo}&cortarNo={cortarNo}&tradTpCd={tradTpCd}&order=point_&showR0=N&page={page}"
        except requests.exceptions.RequestException as e:
            print(f"HTTP 요청 오류 발생: {str(e)}")
            break
        except ValueError:
            print("JSON 디코딩 오류 발생")
            break

    return all_trade_list

def parse_price(price_str):
    price_str = price_str.replace(",", "").strip()
    if "억" in price_str:
        parts = price_str.split("억")
        if len(parts) == 2:
            return float(parts[0]) * 10000 + (float(parts[1].replace("만원", "").strip()) if parts[1].strip() else 0)
        else:
            return float(parts[0]) * 10000
    return float(price_str.replace("만원", "").strip())

def search_listings(sido, gungu, dong, apartment_name, transaction_type, min_price=None, max_price=None, min_area=None, max_area=None):
    transaction_codes = {
        "매매": "A1",
        "전세": "B1",
        "월세": "B2"
    }
    tradTpCd = transaction_codes.get(transaction_type)
    if not tradTpCd:
        print("유효하지 않은 거래 유형입니다. '매매', '전세', '월세' 중 하나를 선택하세요.")
        return []

    hscpNo, cortarNo = find_hscpNo(sido, gungu, dong, apartment_name)
    
    print(hscpNo, cortarNo)

    if hscpNo and cortarNo:
        trade_list = get_trade_list(hscpNo, cortarNo, tradTpCd)

        filtered_list = []

        for trade in trade_list:
            price = parse_price(trade['prcInfo'])
            area = float(trade['spc1']) if trade['spc1'] else 0

            if ((min_price is None or price >= min_price) and
                (max_price is None or price <= max_price) and
                (min_area is None or area >= min_area) and
                (max_area is None or area <= max_area)):
                filtered_list.append(trade)

        return filtered_list
    else:
        return []






In [43]:
if __name__ == "__main__":
    sido = "서울시"
    gungu = "강동구"
    dong = "암사동"
    apartment_name = "강동롯데캐슬퍼스트"
    transaction_type = "매매"
    min_price = 100000  # Minimum price in 만원 (10000 = 1억원)
    max_price = 1000000  # Maximum price in 만원 (100000 = 10억원)
    min_area = 110  # Minimum area in square meters
    max_area = 130  # Maximum area in square meters
    
    listings = search_listings(sido, gungu, dong, apartment_name, transaction_type, min_price, max_price, min_area, max_area)
    if listings:
        df = pd.DataFrame(listings)
        df = df.rename(columns={
            'atclNo': '매물 번호',
            'atclNm': '아파트 이름',
            'bildNm': '동 이름',
            'tradTpNm': '거래 종류',
            'prcInfo': '가격',
            'flrInfo': '층수',
            'spc1': '면적',
            'atclFetrDesc': '특징',
            'tagList': '태그'
        })
        df['가격'] = df['가격'].apply(parse_price)
        df_sorted = df.sort_values(by='가격', ascending=True)
        print(df_sorted[['매물 번호', '아파트 이름', '동 이름', '거래 종류', '가격', '층수', '면적', '특징', '태그']])
        print(f"\n전체 매물 갯수: {len(listings)}")
    else:
        print("매물을 찾을 수 없습니다.")


         매물 번호     아파트 이름  동 이름 거래 종류        가격     층수      면적  \
17  2424262146  강동롯데캐슬퍼스트  137동    매매  129500.0   저/31  112.47   
9   2424835314  강동롯데캐슬퍼스트  103동    매매  132000.0   저/28  112.47   
12  2425566459  강동롯데캐슬퍼스트  107동    매매  135000.0   중/33  112.47   
23  2420250325  강동롯데캐슬퍼스트  103동    매매  135000.0   중/28  112.47   
3   2425354399  강동롯데캐슬퍼스트  105동    매매  135000.0   고/33  112.47   
4   2425424765  강동롯데캐슬퍼스트  107동    매매  135000.0   중/33  112.47   
5   2425425850  강동롯데캐슬퍼스트  128동    매매  135000.0   고/34  112.47   
7   2424969049  강동롯데캐슬퍼스트  128동    매매  135000.0   고/34  112.47   
14  2423526541  강동롯데캐슬퍼스트  139동    매매  135000.0   저/33     113   
24  2423724684  강동롯데캐슬퍼스트  128동    매매  135000.0   고/34  112.47   
21  2422589103  강동롯데캐슬퍼스트  102동    매매  138000.0   고/28     113   
0   2425653135  강동롯데캐슬퍼스트  102동    매매  138000.0  23/28     113   
2   2425603274  강동롯데캐슬퍼스트  102동    매매  138000.0   고/28     113   
1   2425695783  강동롯데캐슬퍼스트  102동    매매  138000.0  23/28     113   
25  242342